# Tutorial 4: Decision Trees and Random Forests 

In this tutorial, we will look at the Decision Tree and Random Forsts algorithms in `scikit-learn`, and assess some of the advantages and disadvantages of those. As Decision Trees are the building blocks of the Random Forest, we will first look into their properties, which gives us insight into most of the Random Forests's hyper-parameters and attributes, and which also shows why using a single Decision Tree is not a great idea. Then, we look into Random Forests and how these are improving on some of the drawbacks of decision trees.

# Part 1: Load and pre-process data
As in the previous tutorials, we will load the data, select the features and split them into *training* and the *validation* sets to understand better how the hyper-parameters may influence overfitting, underfitting and randomness of the algorithm. For tuning, we will however use *cross-validation*.

Since decision trees and random forests are algorithms that process all features *separately* (by splitting along different dimentions), we will not scale the features in this tutorial and use **all** features. This makes the results easier to interpret. 

In [6]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from lib_file import *

## Step 1.1: Load packages and data
As we will use unscaled data, we should load the *training_shuffled.csv* dataset which we have prepared in Tutorial 2:

In [8]:
## TASK: Load the data from the correct file
data = pd.read_csv( 'training_shuffled.csv' , index_col = 0 )

## Step 1.2: Split features and target

In [ ]:
target_name   = 'tilted_radiation'
features_list = [ 'roof_tilt', 'roof_aspect', 'roof_area', 'roof_x', 'roof_y', 'SIS', 'aspect_sign',
                  'SISDIR', 'DHI', 'horizon_S', 'horizon_SSW', 'horizon_SWW', 'horizon_W',
                  'horizon_NWW', 'horizon_SSE', 'horizon_SEE', 'horizon_E', 'horizon_NEE',]

In [ ]:
y = # TASK: EXTRACT TARGET
X = # TASK: EXTRACT FEATURES

## Step 1.3: Split into training and validation set

In [ ]:
X_train, X_val, y_train, y_val = ## TASK: SPLIT THE DATA into 80% training and 20% validation data

# Part 2 - Decision Tree
Random Forests are an *ensemble* of decision trees, which in themselves are what we call *weak learners*. In Machine Learning practice, we will rarely use Decision Trees directly as model, but understanding how these work is very useful for understanding how a Random Forest works.

The Decision Tree for regression is imported as:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

**NOTE**: The documentation can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

## Step 2.1: Score decision tree
Let's start by declaring a default decision tree and obtaining its training and validation scores:

In [ ]:
decision_tree = # TASK: declare default decision tree
# TASK: fit the tree with training data

In [ ]:
# TASK: PRINT THE TRAINING SCORE
# TASK: PRINT THE VALIDATION SCORE

## *Questions*
- a. What do you notice when you fit and score the model multiple times in a row? How does the fitting behaviour (not the performance) of the Decision Tree vary from the models we have seen before?
- b. How does the validation score compare to the default and tuned KNN and SVR algorithms?
- c. How do you explain the training score?

## Step 2.2: Visualising a decision tree
To understand how decision trees work, it is very useful to visualise them. Scikit-learn includes a function to create graphs of decision trees, using the `graphviz` package. You can either create your own graphs using the optional code shown below, or simply display the image *tree.png* that is provided.

In [ ]:
from sklearn.tree import export_graphviz 
from IPython.display import Image

Graphs of decision trees can get very messy and difficult to understand. So, in order to visualise them, we will export only the first three "layers" of the tree (defined by the `max_depth`) to the file *tree.dot*:

In [ ]:
export_graphviz(decision_tree, max_depth = 3, out_file="tree.dot", feature_names=features_list, filled=True)

[OPTIONAL]: If you want to create your own tree from your data and play with the functions of the graphviz exporting function, you need to install the `graphviz` package in your anaconda environment (as you have installed `scikit-learn` and the other packages at the beginning of this tutorial). With graphviz installed, you can run the following command to convert your *tree.dot*-file into an image in png format: 

In [ ]:
# [OPTIONAL]
# Make sure you installed graphviz (exclamation mark is for shell commands)
# Convert dot file to png file.
!dot -Tpng tree.dot -o tree.png

**NOTE**: If you have not installed graphviz, the above task will fail, but you can load the tree.png that we have provided. This was generated from a different tree and a different train_test_split than your current notebook, but it is based on the same dataset.

Now, we can simply display the image (note that this will be the default image if you have not created your own *tree.dot*-file):

In [ ]:
Image(filename='tree.png') 

## *Questions*
- a. What do the 4 different lines in each box represent? In particular, what does the "value" show?
- b. How many distinct features are shown in the tree? How do they compare to the best feature selected in Tutorial 2?
- c. Which one is the first "best split"? Can you explain it in intuitive terms?
- d. What does the tree show about the "smoothness" of the prediction of a decision tree? How this aspect be improved?

## Step 2.3: Feature importance in Decision Trees
As we have seen in Step 2.2, the splitting behaviour of a tree can give us some intuition on which features are important (i.e. which features frequently cause a "best split"). This importance can also be quantified. One measure for feature importance is the _"Gini Impurity"_. This measure is one of the attributes of each decision tree. That means that we can access it, and investigate the importance of each feature:

In [ ]:
feature_importance_tree = pd.Series( decision_tree.feature_importances_, index = features_list )
feature_importance_tree

A good way of visualising the feature importance is a horizontal bar chart, ideally in descending order. This can be done easily with pandas, once the `matplotlib` library is loaded:

In [ ]:
import matplotlib.pyplot as plt # import matplotlib
 # activate interactive plots
%matplotlib notebook           

In [ ]:
feature_importance_tree.sort_values().plot.barh()
plt.tight_layout()

## *Questions*
- a. How do the most important features differ from the most important features based on the mutual information criterion?
- b. Is there a link between the feature importance and the visualisation of the tree in Step 2.3? How do you explain  similarities and differences?
- c. What happens to the feature importances if you re-train a decision tree with exactly the same data?

## Step 2.4: Decision Tree hyper-parameters
In this tutorial, we will have a look at three hyper-parameters that usually have a strong impact on the performance of decision trees (but feel free to test more of the hyper-parameters indicated in the documentation). We will look at:
- The maximum number of features considered at each split (`max_features`)
- The number of nodes in each leaf of the tree (`min_samples_leaf`)
- The tree depth (`max_depth`)

In addition to the over- and underfitting behaviour for the different parameters, we will also look at another aspect that is particular to trees and forests: the **_randomness_**.

#### Max_features
The `max_features` parameter measures the degree of randomness in the model, since it selects randomly, at every split, the features that are considered for splitting the data. By default, all features are considered, so the randomness is minimized. So let's see what happens if we maximise randomness by considering only 1 feature at every split:

In [ ]:
decision_tree = # TASK: DECLARE TREE WITH ONE FEATURE FOR SPLITTING
# TASK: FIT THE TREE

# TASK: PRINT THE TRAINING SCORE
# TASK: PRINT THE VALIDATION SCORE

## *Questions*
- a. How does the model perform with only one feature?
- b. Another recommended value for `max_features` is the square-root of the number of features. How does the model in this case?
- c. Does this hyper-parameter change the over/underfitting behaviour of the default decision tree?
- [OPTIONAL] d. Create a graph for the tree fitted with `max_features = 1`. What do you notice about the features in the nodes of the graph?

#### Min_samples_leaf
The `min_samples_leaf` defines how many samples need to be present, at minimum, in each leaf node of the tree. This means that a node of the tree is no longer considered for splitting if `min_samples_leaf` or less training samples are in that node. As we have seen, the default `min_samples_leaf` is 1 and leads to over-fitting (a training score of 1). So, let's see what happens when we change the `min_samples_leaf` to higher values (let's start with 10):

**NOTE**: The behviour and effect of the `min_samples_leaf` hyper-parameter is similar to that of `min_samples_split`, so here we do not look at both hyper-parameters, but both can be tuned.

In [ ]:
decision_tree = # TASK: DECLARE TREE WITH AT LEAST 10 SAMPLES PER LEAF
# TASK: FIT THE TREE

# TASK: PRINT THE TRAINING SCORE
# TASK: PRINT THE VALIDATION SCORE

## *Questions*
- a. How does the performance change with respect to the default model?
- b. What happens if you increase `min_samples_leaf` further (let's say to 100)?
- c. [OPTIONAL]: What is the optimal value of `min_samples_leaf`?

#### Max_depth
The `max_depth` parameter determines, how "developled" a model is, i.e. how many layers it can contain (see the image *tree.png* in step 2.2). This is closely related to the `min_samples_leaf` - parameter, as it is another parameter that determines how much growing of the tree is "allowed". The default is to let it grow fully, until all data is split as much as possible (which depends on the minimum samples for splits and in the leaves), but you can set it to any integer value:

In [ ]:
decision_tree = # TASK: DECLARE TREE WITH SOME MAXIMUM DEPTH
# TASK: FIT THE TREE

# TASK: PRINT THE TRAINING SCORE
# TASK: PRINT THE VALIDATION SCORE

## *Questions*
- a. Which values of `max_depth` lead to over-fitting, which values lead to underfitting?
- b. [OPTIONAL]: What is the optimal value of `max_depth`?

# Part 3 - Random Forests
Random Forests are __ensembles__ of decision trees, i.e. a collection of these trees. To avoid a simple repetition, trees are trained on random subsets of the data, which are sampled *with replacement* from the training data as part of the fitting process. The ensemble prediction of these estimators is often more robust to small changes in hyper-parameters, which makes them suitable for "quick and dirty" Machine Learning - it works well in many cases without much training. 

The documentation is found at: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
from sklearn.ensemble import RandomForestRegressor

Let's train a default Random Forest and see its result:

In [ ]:
# TASK: declare, fit and score Random Forest Regressor

As the training process of each tree in the random forest is based on a random subset of the data, some samples are not included in the training of each of the trees (see the concept of "bagging"). The prediction of a tree on these samples makes up the "out-of-bag"-error, which is another measure on the algorithm's performance on unseen data. The OOB score is an attribute of a random forest, and the computation of the OOB score has to be set as an argument of the `RandomForestRegressor` by setting `oob_score = True`:

In [ ]:
rf = # TASK: declare a random forest with oob_score = True
# TASK: fit the random forest

Then, the OOB score is obtained as:

In [ ]:
rf.oob_score_

## *Questions*
- a. How does the default behaviour compare to previous algorithms?
- b. Why is the default training score no longer equal to 1 (as it was in the decision tree), despite equal default hyper-parameters?
- c. How does the OOB score compare to the validation score?

## Step 3.1: Feature importance of Random Forests
As in the decision tree, the random forest also has a *feature importance* measure, which can help in the analysis and selection of features. Following the example from Step 2.3, we can obtain it as:

In [ ]:
importances = ## TASK: COMPUTE FEATURE IMPORTANCES

The feature importance of the forest is nothing else than the average of the feature importances of each tree. The attributes of each individual tree are also stored in the random forest, which means that we can also access these. That can be very helpful for many purposes, such as the estimation of modelling uncertainty. 

The trees are stored in the `estimators_`-attribute as a list. So, just to practice, let's display the first decision tree of the random forest:

In [ ]:
## TASK: Display the first decision tree of the random forest

Since the feature importance of the forest is a mean, this means that there is also a *standard deviation* belonging to it. To obtain the standard deviation, we have to access the feature importance scores of each tree:

In [ ]:
# compute the standard deviation of the feature importance
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

##### Evaluation of feature importance

Similar to individual trees, we can display the feature importance for the forest, but now we do not only have the mean, we can also show the std. deviation.

**TASK**: Create a dataframe with feature importance and its standard deviation as columns and the feature names as index, sorted in descending order. Name the columns **Feature importance** and **Std. deviation**.

In [ ]:
feature_importance = # TASK: Create dataframe (see above)
# Show dataframe 

To the plot from step 2.2, we can now add the error bars corresponding to the field 'Std. deviation':

In [ ]:
feature_importance.sort_values('Feature importance').plot.barh(xerr = 'Std. deviation', capsize = 2)
plt.tight_layout()

## *Questions*
- a. Are there any differences between the feature importance of the Random Forest and of the Decision Tree? If so, what are these differences?
- b. Does the feature importance change if you re-train the Random Forest with the same data?

## Step 3.2: Hyper-parameter tuning

Now that we have looked into some of the Random Forest's key properties and attributes, let's focus on the hyper-parameter tuning. Most of the hyper-parameters of the Random Forest are equal to those of decision trees - in fact, they are the parameters that are passed to each tree in the forest. Let's see how these parameters affect the ensemble prediction, compared to the prediction of individual trees.

For this, we will look at the performance with a very high degree of randomness, namely with `max_features = 1`, and a hyper-parameter that previously improved the model performance significantly, namely `min_samples_leaf = 10`. 

**_TASK 1_**: For both cases (`max_features = 1` and `min_samples_leaf = 10`), declare a random forest and print the training and validation scores.

**_TASK 2_**: Perform a hyper-parameter tuning based on GridSearchCV for all hyper-parameters from Step 2.4 (`max_features`, `min_samples_leaf`, `max_depth`) following the approach from Tutorial 3 using **5-fold cross-validation**.

*HINT 1*: Don't forget to import the necessary functions<br>
*HINT 2*: You can set the parameter n_jobs = -1 in the default Random Forest to enable paralellised training

## *Questions*
- a. Why is the performance of the model with `max_features = 1` improved by such a large margin compared to Decision Trees? What does this suggest about the robustness of the random forest to changes in the hyper-parameters?
- b. How does the performance of the random forest with `min_samples_leaf = 10` compare to the default, and how do you interpret this change in comparison to the behaviour observed for decision trees?
- c. What are the best parameters and the best cross-validation score that you can obtain?

## Step 3.3: Tuning n_estimators
The hyper-parameter which is intrinsic to Random Forests is the number of trees in the forest, called `n_estimators` (default 100). What differs between `n_estimators` and "normal" hyper-parameters is that the impact of the number of estimators on the model performance follows the motto _"the more the merrier"_ - more trees means more random instances to average across, which results in a more stable performance, in general.

To see the effect of `n_estimators` on the model performance and time, let's test a range of values between 1 and an arbitrary maximum of 5000:

In [ ]:
n_estimators = [1, 2, 3, 5, 10, 20, 50, 100, 200, 500, 1000, 2000]

To test the entire range of values and obtain the entire fitting and validation log, we will again use the `GridSearchCV`:

**TASK**: Perform a Grid Search on the number of estimators, using the best estimator from Step 3.2 as default model.

As seen in Tutorial 3, we can convert the results into a dataframe, which we can use for visualisation and plotting:

In [ ]:
rf_stats = # TASK: Convert the cv results into a dataframe 

In [ ]:
# TASK: Show the first 3 lines of the new dataframe

#### Visualisation
We can also plot the mean and standard deviation of the cross-validation score as a line plot. We can even split the plot into two axes (a left and a right one), to show both the score and the required fitting and scoring times simultaneously and in a comparable fashion. 

To do so, we set the `param_n_estimators` as the index of our dataframe `rf_stats`, so that pandas automatically uses the right x-axis:

In [ ]:
rf_stats = rf_stats.set_index('param_n_estimators') # Set index of the DataFrame rf_stats

In [ ]:
plt.figure(figsize = (10,3.5))

# Left axis: Cross-validation score and formatting of plot, including the logarithmic scale of the x-axis
rf_stats.mean_test_score.plot(yerr = rf_stats.std_test_score, label = 'CV score', capsize = 2)
plt.ylabel('Cross-validation score ($R^2$)')
plt.xscale('log')
plt.legend()
plt.grid(axis = 'y', ls = '--')

# Right axis: Time and formatting
plt.gca().twinx()
rf_stats.mean_fit_time.plot(yerr = rf_stats.std_fit_time, label = 'Fit time', c = 'C1', capsize = 2)
rf_stats.mean_score_time.plot(yerr = rf_stats.std_score_time, label = 'Score time', c = 'C2', capsize = 2)
plt.ylabel('Time (s)')
plt.legend(loc = 4)

plt.tight_layout()

**NOTE**: The x-axis is on a logarithmic scale!

## *Questions*
- a. How do you assess the trade-off between performance and time for this dataset? What if you change the scale of the x-axis from linear to logarithmic?
- b. Which value of n_estimators would you choose and why?

## Step 3.4: Model testing
Finally, for comparison with the models from Tutorial 3, we need to obtain the test score. As we have not been working with standardised data here, we can load the *test_data.csv*.

In [ ]:
test_data = pd.read_csv('test_data.csv', index_col = 0)

In [ ]:
X_test = test_data[ features_list ]
y_test = test_data[ target_name ]

With the test set loaded, predict the labels for `X_test` for the best estimator obtained in 3.3:

In [ ]:
y_test_pred = # TASK: Obtain prediction for the test set from the best estimator obtained previously

Now, compute the R2 and MAE scores for the Random Forest on the test set:

In [ ]:
# TASK: Compute the R2 and MAE for the test set

## *Questions*
- How does the test score compare to that obtained for the KNN and SVR?
- Out of all three models we have seen, which one would you choose for this problem? Why?